In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    AIMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
import os, sys
sys.path.insert(1,"D:\Github\DeepLake-Langchain")
import credentials
os.environ["OPENAI_API_KEY"] = credentials.openai

In [2]:
chat = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)

template = "You are a helpful assistant that translates english to pirate."

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template= "{input}"
human_template_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [3]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_template_prompt])
chain = LLMChain(llm = chat, prompt=chat_prompt)
chain.run("I love Programming")

"I be lovin' the art of code plunderin'"

In [4]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [5]:
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template
)

In [6]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

In [7]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

In [8]:
print(dynamic_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic



Word: sunny
Antonym: gloomy


Word: big
Antonym:


## SemanticSimilarity Selector

In [9]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate


In [10]:
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template = "Input: {input}\nOutput: {output}",
)

In [11]:
examples = [
     {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

In [14]:
os.environ["ACTIVELOOP_TOKEN"]= credentials.active_loop

In [15]:
my_activeloop_org_id = credentials.active_loop_org_id
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


In [16]:
# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [17]:
# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (5, 1536)  float32   None   
    id        text      (5, 1)      str     None   
 metadata     json      (5, 1)      str     None   
   text       text      (5, 1)      str     None   


In [18]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt,
    prefix = "Convert the temperature from Celsius to Fahrenheit",
    suffix = "Input: {temperature}\nOutput:",
    input_variables = ["temperature"],
    )

In [19]:
# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


In [20]:
# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
   text       text      (6, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:


In [21]:
print(similar_prompt.format(temperature="55°C"))

Convert the temperature from Celsius to Fahrenheit

Input: 50°C
Output: 122°F

Input: 55°C
Output:
